# Proceso ETL para Ingenierida de Datos
👉Este script pretende automatizar el proceso de ETL dentro del analisis de varios conjuntos de datos: tanto archivos .csv , como json asi como tambien xml
👉 Además, este script crea un archivo de log, para poder tener control sobre lo que se realiza a cada momento dentro de cada una de las etapas de la carga de los datos

In [2]:
# datos
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
!unzip source.zip

--2024-03-18 21:11:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/source.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2707 (2.6K) [application/zip]
Saving to: ‘source.zip’

source.zip          100%[===================>]   2.64K  --.-KB/s    in 0s      

2024-03-18 21:11:00 (1.25 GB/s) - ‘source.zip’ saved [2707/2707]

Archive:  source.zip
  inflating: source3.json            
  inflating: source1.csv             
  inflating: source2.csv             
  inflating: source3.csv             
  inflating: source1.json            
  infl

In [3]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

In [5]:
log = "archivo_log.txt"
archivo_destino = "datos_transformados.csv"

## Carga de los archivos separados por su extension

In [6]:
def extraer_de_csv(file_to_process):
  dataframe = pd.read_csv(file_to_process)
  return dataframe

In [12]:
def extraer_de_json(file_to_process):
    dataframe = pd.read_json(file_to_process, lines=True)
    return dataframe

**Para extraer de un archivo XML, primero se debe analizar los datos del archivo utilizando
la función ElementTree.
Luego puede extraer información relevante de estos datos y
agregarla a un marco de datos de pandas de la siguiente manera.
Nota: Se Debe conocer los encabezados de los datos extraídos para escribir esta función.
En estos datos, extrae los encabezados "nombre", "altura" y "peso" para diferentes personas**

In [14]:
def extraer_de_xml(file_to_process):
    dataframe = pd.DataFrame(columns=["name", "height", "weight"])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        name = person.find("name").text
        height = float(person.find("height").text)
        weight = float(person.find("weight").text)
        dataframe = pd.concat([dataframe, pd.DataFrame([{"name":name, "height":height, "weight":weight}])], ignore_index=True)
    return dataframe

# Definiendo la funcion Extraer

In [8]:
def extraer():
    # creando dataframe vacio
    datos_extraidos= pd.DataFrame(columns=["name" , "height" , "weight"])

    #procesando archivos csv
    for csvfile in glob.glob("*.csv"):
        datos_extraidos = pd.concat([datos_extraidos, pd.DataFrame(extraer_de_csv(csvfile))] , ignore_index=True)

    # procesando archivos json
    for jsonfile in glob.glob("*.json"):
        datos_extraidos = pd.concat([datos_extraidos, pd.DataFrame(extraer_de_json(jsonfile))] , ignore_index=True)

    # procesando archivos XML
    for xmlfile in glob.glob("*.xml"):
        datos_extraidos = pd.concat([datos_extraidos, pd.DataFrame(extraer_de_xml(xmlfile))] , ignore_index=True)

    # retorno de los datos
    return datos_extraidos

# Definiendo la funcion de transformacion de los datos -> en este caso para poder tener un orden de los mismos

In [16]:
def transformar(datos_extraidos):
    '''Convertir pulgadas a metros y redondear a dos decimales
    1 pulgada son 0,0254 metros '''
    datos_extraidos["height"] = round(datos_extraidos.height * 0.0254,2)

    '''Convertir libras a kilogramos y redondear a dos decimales
     1 libra son 0,45359237 kilogramos'''

    datos_extraidos["weight"]= round(datos_extraidos.weight * 0.45359237,2)

    return datos_extraidos


# Definiendo la funcion de carga de los datos

In [9]:
def carga_datos(archivo_destino, datos_transformados):
    datos_transformados.to_csv(archivo_destino)

# Creando LOG para poder tener control sobre en que momentos se realizo cada operación

In [10]:
def proceso_log(mensaje):
    formato_fecha = '%Y-%h-%d-%H:%M:%S' # formato de fecha año-mes-dia-hora-minuto-segundo
    ahora = datetime.now() # obtener el tiempo actual
    formato_fecha = ahora.strftime(formato_fecha)
    with open(log,"a") as f:
        f.write(formato_fecha + ',' + mensaje + '\n')

# Creando LOG'S

In [17]:
# Log inicio de proceso de ETL
proceso_log("Inicio proceso ETL")

# Log de inicio de la fase de extraccion
proceso_log("Extraccion comenzando")
datos_extraidos = extraer()

# Log de finalizacion de la fase de extraccion
proceso_log("Fase de Extraccion Finalizada")

# Log FASE DE TRANSFORMACION
proceso_log("Fase de Transformacion Iniciada")
datos_transformados = transformar(datos_extraidos)
print("Datos Transformados")
print(datos_transformados)

# Log de terminacion de fase de transformacion
proceso_log("Fase de Transformación Completada!")

# Log de proceso de carga
proceso_log("Fase de proceso de carga iniciada")
carga_datos(archivo_destino,datos_transformados)

# Log de termino fase de carga
proceso_log("Fase de Carga finalizada")

# Log de terminacion de proceso de ETL
proceso_log("Proceso ETL Finalizado con Éxito!")

Datos Transformados
     name  height  weight
0    alex    1.67   51.25
1    ajay    1.82   61.91
2   alice    1.76   69.41
3    ravi    1.73   64.56
4     joe    1.72   65.45
5    alex    1.67   51.25
6    ajay    1.82   61.91
7   alice    1.76   69.41
8    ravi    1.73   64.56
9     joe    1.72   65.45
10   alex    1.67   51.25
11   ajay    1.82   61.91
12  alice    1.76   69.41
13   ravi    1.73   64.56
14    joe    1.72   65.45
15   jack    1.74   55.93
16    tom    1.77   64.18
17  tracy    1.78   61.90
18   john    1.72   50.97
19   jack    1.74   55.93
20    tom    1.77   64.18
21  tracy    1.78   61.90
22   john    1.72   50.97
23   jack    1.74   55.93
24    tom    1.77   64.18
25  tracy    1.78   61.90
26   john    1.72   50.97
27  simon    1.72   50.97
28  jacob    1.70   54.73
29  cindy    1.69   57.81
30   ivan    1.72   51.77
31  simon    1.72   50.97
32  jacob    1.70   54.73
33  cindy    1.69   57.81
34   ivan    1.72   51.77
35  simon    1.72   50.97
36  jacob    1.70 

In [18]:
datos_transformados

,name,height,weight
0,alex,1.67,51.25
1,ajay,1.82,61.91
2,alice,1.76,69.41
3,ravi,1.73,64.56
4,joe,1.72,65.45
5,alex,1.67,51.25
6,ajay,1.82,61.91
7,alice,1.76,69.41
8,ravi,1.73,64.56
9,joe,1.72,65.45
